In [1]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline

from sklearn.model_selection import RandomizedSearchCV

import datetime

import matplotlib.pyplot as plt

import holidays

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor


In [2]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")


In [3]:
# Add external data : weather data
weather = pd.read_csv(
    "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/weather_data.csv.gz",
    parse_dates=["AAAAMMJJHH"],
    date_format="%Y%m%d%H",
    compression="gzip",
    sep=";",
).rename(columns={"AAAAMMJJHH": "date"})

weather = weather[
    (weather["date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (weather["date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

weather_reduced = (
    weather.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "QDXI3S"])
    .groupby("date")
    .mean()
    .dropna(axis=1, how="all")
    .interpolate(method="linear")
)

weather_reduced = (
    weather_reduced
    .drop(columns=[
        "PSTAT", "DD", "PMER", "PMERMIN", "QNEIGETOT", "QTCHAUSSEE", "ALTI", "QDRR1", "DXY", "FXY",
        "QTNSOL", "QPMER", "DXI", "QFF", "QGLO2", "QGLO", "FF", "QHFXI3S", "QINS2", "QINS",
        "QFXI3S", "RR1", "NEIGETOT", 'HXI', 'HFXI3S', "HTN", "HTX", "HUN", "HUX", "FXI3S",
        "T10", "T20", "T50", "T100", "TNSOL", "TN50", "TCHAUSSEE", "TN", "TX"
    ])
    .dropna(axis=1, how="all")
    .loc[:, weather_reduced.nunique(dropna=True) > 1]
    .drop(columns=["QTD", "QTN", "QUN", "QUX", "QTSV", "QTX", "GLO2", "INS2", "UN", "UX"])
)

# We merge :
df_train = df_train.merge(weather_reduced, left_on="date", right_on="date", how="left")
df_test = df_test.merge(weather_reduced, left_on="date", right_on="date", how="left")

In [4]:
# Add jour ferie data
jour_feries = (
    pd.read_csv(
        "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/jours_feries_metropole.csv",
        date_format="%Y%m%d%H"  # Ensure date format is handled correctly
    )
    .drop(columns=["annee", "zone"])  # Drop unnecessary columns
)

# Convert 'date' column to datetime
jour_feries['date'] = pd.to_datetime(jour_feries['date'])

# Filter rows based on the date range of df_train and df_test
jour_feries = jour_feries[
    (jour_feries["date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (jour_feries["date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

In [5]:
# Add mouvements sociaux data :
mouvements_sociaux = (
    pd.read_csv(
        "/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/mouvements-sociaux-depuis-2002.csv",
        date_format="%Y%m%d%H",
        sep=";"
    )
    .drop(columns=['date_de_fin', 'Organisations syndicales', 'Métiers ciblés par le préavis',
                   'Population devant travailler ciblee par le préavis', 'Nombre de grévistes du préavis'])  # Drop unnecessary columns
)

mouvements_sociaux['Date'] = pd.to_datetime(mouvements_sociaux['Date'])

mouvements_sociaux = mouvements_sociaux[
    (mouvements_sociaux["Date"] >= df_train["date"].min() - datetime.timedelta(hours=1))
    & (mouvements_sociaux["Date"] <= df_test["date"].max() + datetime.timedelta(hours=1))
]

mouvements_sociaux = mouvements_sociaux[mouvements_sociaux['Date'] != pd.Timestamp('2021-03-08')]

In [6]:
# Extract the date feature on different time scales :
fr_holidays = holidays.France()

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Add a feature to indicate if the day is a holiday in France
    X["is_holiday"] = X["date"].apply(lambda d: 1 if d in fr_holidays else 0)

    # and if it is a jour ferie in France :
    X["is_jour_ferie"] = X["date"].dt.date.isin(jour_feries['date']).astype(int)

    # and it is a jour of "mouvement social" in France :
    X["is_jour_mouvement_social"] = X["date"].dt.date.isin(mouvements_sociaux['Date']).astype(int)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

df_train = _encode_dates(df_train)
df_test = _encode_dates(df_test)


In [7]:
# Preprocessing :

# Extract features from counter_installation_date
for df in [df_train, df_test]:
    df["installation_year"] = df["counter_installation_date"].dt.year
    df["installation_month"] = df["counter_installation_date"].dt.month

df_train = df_train.drop(columns=["counter_installation_date"])
df_test = df_test.drop(columns=["counter_installation_date"])

# Label encode high-cardinality categorical features
label_encoders = {}
for col in ["counter_id", "site_id", "counter_name", "site_name", "counter_technical_id", "coordinates"]:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.fit_transform(df_test[col])
    label_encoders[col] = le

In [8]:
df_train.head()

,counter_id,counter_name,site_id,site_name,bike_count,coordinates,counter_technical_id,latitude,longitude,log_bike_count,...,month,day,weekday,hour,is_weekend,is_holiday,is_jour_ferie,is_jour_mouvement_social,installation_year,installation_month
0,1,10,0,5,0.0,10,0,48.846028,2.375429,0.000000,...,9,1,1,2,0,0,0,0,2013,1
1,1,10,0,5,1.0,10,0,48.846028,2.375429,0.693147,...,9,1,1,3,0,0,0,0,2013,1
2,1,10,0,5,0.0,10,0,48.846028,2.375429,0.000000,...,9,1,1,4,0,0,0,0,2013,1
3,1,10,0,5,4.0,10,0,48.846028,2.375429,1.609438,...,9,1,1,15,0,0,0,0,2013,1
4,1,10,0,5,9.0,10,0,48.846028,2.375429,2.302585,...,9,1,1,18,0,0,0,0,2013,1


In [9]:
df_train.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'coordinates', 'counter_technical_id', 'latitude', 'longitude',
       'log_bike_count', 'DRR1', 'HXY', 'FXI', 'T', 'QT', 'TD', 'DG', 'U',
       'QU', 'DHUMI40', 'DHUMI80', 'TSV', 'VV', 'WW', 'GLO', 'INS', 'year',
       'month', 'day', 'weekday', 'hour', 'is_weekend', 'is_holiday',
       'is_jour_ferie', 'is_jour_mouvement_social', 'installation_year',
       'installation_month'],
      dtype='object')

In [10]:
'''ArithmeticError# Creation of bins for the temperature feature : less than 0 degrees, between 0 and 5, between 5 and 15, between 15 and 30, more than 30
bins = [-float('inf'), 0, 5, 15, 30, float('inf')]
labels = ["< 0°C", "0-5°C", "5-15°C", "15-30°C", "> 30°C"]

# Create a new categorical feature for temperature
df_train['temp_category'] = pd.cut(df_train['T'], bins=bins, labels=labels)
df_test['temp_category'] = pd.cut(df_test['T'], bins=bins, labels=labels)

# One-hot encode the categories for the model
df_train = pd.get_dummies(df_train, columns=['temp_category'], drop_first=True).astype(int)
df_test = pd.get_dummies(df_test, columns=['temp_category'], drop_first=True).astype(int)
'''


'ArithmeticError# Creation of bins for the temperature feature : less than 0 degrees, between 0 and 5, between 5 and 15, between 15 and 30, more than 30\nbins = [-float(\'inf\'), 0, 5, 15, 30, float(\'inf\')]\nlabels = ["< 0°C", "0-5°C", "5-15°C", "15-30°C", "> 30°C"]\n\n# Create a new categorical feature for temperature\ndf_train[\'temp_category\'] = pd.cut(df_train[\'T\'], bins=bins, labels=labels)\ndf_test[\'temp_category\'] = pd.cut(df_test[\'T\'], bins=bins, labels=labels)\n\n# One-hot encode the categories for the model\ndf_train = pd.get_dummies(df_train, columns=[\'temp_category\'], drop_first=True).astype(int)\ndf_test = pd.get_dummies(df_test, columns=[\'temp_category\'], drop_first=True).astype(int)\n'

In [11]:
X_train = df_train.drop(columns=["bike_count", "log_bike_count"])
y_train = df_train["log_bike_count"]

X_test = df_test.copy()

In [12]:
# Define the XGBoost model
xgb_model = xgb.XGBRegressor()

# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9, 12],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

In [13]:
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Perform the search
random_search.fit(X_train, y_train)

# Get the best parameters and score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", -random_search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
Best Score: 0.8906274546492916


In [14]:
# Update the model with the best parameters
best_xgb_model = random_search.best_estimator_

# Initialize the XGBoost regressor
model = best_xgb_model

# Fit the model
model.fit(
    X_train, y_train,
)

# Make Predictions on Test Data
y_predictions = model.predict(X_test)

In [15]:
'''
# Initialize the XGBoost regressor
model = XGBRegressor(
    # objective="reg:squarederror",  # Use squared error for regression
    max_depth=6,                  # Maximum depth of the trees
    learning_rate=0.1,            # Step size shrinkage
    n_estimators=500,             # Number of boosting rounds
    subsample=0.8,                # Fraction of samples for training each tree
    colsample_bytree=0.8,         # Fraction of features for each tree
    random_state=42,              # Reproducibility
)

# Fit the model
model.fit(
    X_train, y_train,
)

# Make Predictions on Test Data
y_predictions = model.predict(X_test)
'''


'\n# Initialize the XGBoost regressor\nmodel = XGBRegressor(\n    # objective="reg:squarederror",  # Use squared error for regression\n    max_depth=6,                  # Maximum depth of the trees\n    learning_rate=0.1,            # Step size shrinkage\n    n_estimators=500,             # Number of boosting rounds\n    subsample=0.8,                # Fraction of samples for training each tree\n    colsample_bytree=0.8,         # Fraction of features for each tree\n    random_state=42,              # Reproducibility\n)\n\n# Fit the model\nmodel.fit(\n    X_train, y_train,\n)\n\n# Make Predictions on Test Data\ny_predictions = model.predict(X_test)\n'

In [16]:
print(y_predictions)

[0.406196  1.4767338 2.0724719 ... 5.172175  4.7162175 3.6701746]


In [17]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_XGboost_vsimple_weather_newdata.csv", index=False)

In [18]:
# We can print the RMSE on the training data :
y_train_predictions = model.predict(X_train)
rmse_train = mean_squared_error(y_train, y_train_predictions, squared=False)
print(f"Training RMSE: {rmse_train}")

Training RMSE: 0.32663356896716444


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
'''
# code to get feature importance :


# Step 1: Extract the preprocessor and feature names
# Retrieve the preprocessor from the pipeline
preprocessor = pipeline.named_steps['preprocessor']

# Get the feature names after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Step 2: Extract the trained XGBoost model and feature importance
xgb_model = pipeline.named_steps['model']

# Get feature importances from the trained XGBoost model
feature_importance = xgb_model.feature_importances_

# Step 3: Combine feature names and importance scores into a DataFrame
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Sort features by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features by Importance:")
importance_df.head(40)
'''

'\n# code to get feature importance :\n\n\n# Step 1: Extract the preprocessor and feature names\n# Retrieve the preprocessor from the pipeline\npreprocessor = pipeline.named_steps[\'preprocessor\']\n\n# Get the feature names after preprocessing\nfeature_names = preprocessor.get_feature_names_out()\n\n# Step 2: Extract the trained XGBoost model and feature importance\nxgb_model = pipeline.named_steps[\'model\']\n\n# Get feature importances from the trained XGBoost model\nfeature_importance = xgb_model.feature_importances_\n\n# Step 3: Combine feature names and importance scores into a DataFrame\nimportance_df = pd.DataFrame({\'Feature\': feature_names, \'Importance\': feature_importance})\n\n# Sort features by importance\nimportance_df = importance_df.sort_values(by=\'Importance\', ascending=False)\n\n# Display top features\nprint("Top Features by Importance:")\nimportance_df.head(40)\n'